In [1]:
from rubiks import *
from solver import *
from heuristics import *
import timeit
import pandas as pd
import dataframe_image as dfi
import pickle

In [2]:
h = {
    "adapted-CFOP": adaptedCFOP,
    "adapted-CFOP2": adaptedCFOP2,
    "numberOfSolidRows": numberOfSolidRows,
    "numberOfSolidFaces": numberOfSolidFaces,
    "numberOfSolidRowsNFace": numberOfSolidRowsWithSolidFace
}

In [3]:
results = {}

In [4]:
all_steps = [3,5,7,10]
eps = 100
scrambles = {steps: {} for steps in all_steps}

# make scrambles
for steps in all_steps:
    for i in range(eps):
        scramble = generateScramble(steps)
        scrambles[steps][scramble] = None
    

In [5]:
results  = {name: {} for name in h}
for (name, heuristic) in h.items():
    result = {}
    for step, step_scrambles in scrambles.items():
        time = 0
        solved = 0
        for scramble in step_scrambles:
            c = cube()
            c.move(scramble)

            s = solver(c, h=heuristic)
            start = timeit.default_timer()
            s.solve(maxDepth=10, display=False, verbose=False, earlyTermination=500)
            if s.solutionFound:
                solved +=1
            stop = timeit.default_timer()
            timeTaken = stop - start
            time += timeTaken

        avg_time = time/len(step_scrambles)
        result['{}_slice_avg_time'.format(step)] = round(avg_time, 2)
        result['{}_slice_solves (%)'.format(step)] = round((solved/len(step_scrambles)) * 100, 2)
        results[name] = result
        

In [6]:
df = pd.DataFrame.from_dict(results, orient='index')
df

,3_slice_avg_time,3_slice_solves (%),5_slice_avg_time,5_slice_solves (%),7_slice_avg_time,7_slice_solves (%),10_slice_avg_time,10_slice_solves (%)
adapted-CFOP,0.01,82.83,0.07,58.0,0.35,29.0,0.68,12.0
adapted-CFOP2,0.01,86.87,0.06,59.0,0.27,33.0,0.52,7.0
numberOfSolidRows,0.01,84.85,0.09,54.0,0.30,23.0,0.47,7.0
numberOfSolidFaces,0.02,86.87,0.11,45.0,0.27,12.0,0.33,1.0
numberOfSolidRowsNFace,0.01,84.85,0.09,51.0,0.29,25.0,0.52,5.0


In [7]:
n = 4
a = df[df.columns[:n]].values
adf = pd.DataFrame(a)
adf.rename(
    index={i: v for i, v in enumerate(list(df.index))},
    columns={i: v for i, v in enumerate(list(df.columns)[:n])},
    inplace=True)
adf

,3_slice_avg_time,3_slice_solves (%),5_slice_avg_time,5_slice_solves (%)
adapted-CFOP,0.01,82.83,0.07,58.0
adapted-CFOP2,0.01,86.87,0.06,59.0
numberOfSolidRows,0.01,84.85,0.09,54.0
numberOfSolidFaces,0.02,86.87,0.11,45.0
numberOfSolidRowsNFace,0.01,84.85,0.09,51.0


In [12]:
dfi.export(adf,"../images/results/scores1.png")

In [9]:
n = 4
b = df[df.columns[n:]].values
bdf = pd.DataFrame(b)
bdf.rename(
    index={i: v for i, v in enumerate(list(df.index))},
    columns={i: v for i, v in enumerate(list(df.columns)[n:])},
    inplace=True)
bdf

,7_slice_avg_time,7_slice_solves (%),10_slice_avg_time,10_slice_solves (%)
adapted-CFOP,0.35,29.0,0.68,12.0
adapted-CFOP2,0.27,33.0,0.52,7.0
numberOfSolidRows,0.30,23.0,0.47,7.0
numberOfSolidFaces,0.27,12.0,0.33,1.0
numberOfSolidRowsNFace,0.29,25.0,0.52,5.0


In [11]:
dfi.export(bdf,"../images/results/scores2.png")